In [1]:
from transformers import BertTokenizer, BertForMaskedLM, TrainingArguments, Trainer
from google.cloud import bigquery
import torch
import pandas as pd

lm = 'indobenchmark/indobert-base-p1'
tokenizer = BertTokenizer.from_pretrained(lm)
model = BertForMaskedLM.from_pretrained(lm)

Some weights of BertForMaskedLM were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [3]:
%%bigquery products --use_bqstorage_api
SELECT id, name, description, weight, shop_id, price, menu_name, main_category, sub_category
FROM `food-id-app.external_data_temp.EXTERNAL_PRODUCTS`
WHERE main_category = 'Produk Susu'

In [4]:
# products = pd.read_csv('../data/pareto_training.csv')
products = products.dropna(subset=['name'])
text = products['name'].values

In [5]:
inputs = tokenizer(text.tolist(), return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [6]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [7]:
rand = torch.rand(inputs.input_ids.shape)

mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [8]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

In [9]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [10]:
class MLMDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings.input_ids)
    
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

dataset = MLMDataset(inputs)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [12]:
args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=32,
    num_train_epochs=2,
    save_steps = 3000
)

In [13]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

trainer.train()

/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Step,Training Loss
500,0.171900
1000,0.017400
1500,0.013600
2000,0.012000
2500,0.010900
3000,0.010300
3500,0.009800
4000,0.009300
4500,0.009500
5000,0.008800


/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construc

RuntimeError: [enforce fail at inline_container.cc:298] . unexpected pos 153640832 vs 153640784

In [1]:
model.save_pretrained("../models/mlm_susu")

NameError: name 'model' is not defined